In [3]:
#Importing some of the Essential Libraries
import pandas as pd #Used for Data Manipulation and analysis
import numpy as np #Used for numerical and mathematical operations
from tensorflow import keras #Used to build and train deep learning models.
from tqdm import tqdm #Used to create a progress bars in python.
import nltk #Used for text processing, including tokenization, stemming, and more.

In [2]:
!pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/79/33/67c4ed826f5227655225c3feaaecd15afb8453e827334ddae95a7fba07ac/regex-2023.10.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.0/777.0 kB 60.2 MB/s eta 0:00:00


In [4]:
print(keras.__version__)

2.12.0


In [5]:
train = pd.read_csv('yelp_review_database/train.csv', names = ['sentiment', 'text'] )
test  = pd.read_csv('yelp_review_database/test.csv',  names = ['sentiment', 'text'] )

In [6]:
train = train[:15000]
train.head()


,sentiment,text
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [7]:
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Dropout

In [8]:
# Just for example
max_features = 2000
Encoder = keras.layers.experimental.preprocessing.TextVectorization( max_tokens = max_features)
Encoder.adapt(train['text'].values)

vocab = np.array(Encoder.get_vocabulary())
print(vocab[:20])

example ="This is an example to test the encoder that we just created!"
print(Encoder(example).numpy())
print(" ".join(vocab[Encoder(example).numpy()]))

['' '[UNK]' 'the' 'and' 'i' 'a' 'to' 'was' 'of' 'it' 'is' 'in' 'for'
 'that' 'my' 'this' 'but' 'with' 'they' 'you']
[  15   10   56 1968    6 1913    2    1   13   23   39    1]
this is an example to test the [UNK] that we just [UNK]


In [9]:
max_features = 2000
tokenizer = Tokenizer(num_words = max_features, )
tokenizer.fit_on_texts(train['text'].values)
X = tokenizer.texts_to_sequences(train['text'].values)
X = pad_sequences(X, padding = 'post' ,maxlen=300)
Y = pd.get_dummies(train['sentiment']).values

vocab_size = len(tokenizer.word_index)+1

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.25, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(11250, 300) (11250, 2)
(3750, 300) (3750, 2)


In [11]:
from tqdm import tqdm
embedding_vector = {}
f = open('glove.6B.50d.txt')
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

400000it [00:05, 74155.78it/s]


In [12]:
embedding_matrix = np.zeros((vocab_size,50))
for word,i in tqdm(tokenizer.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

100%|██████████| 37637/37637 [00:00<00:00, 501706.73it/s]


In [13]:
embedding_matrix.shape

(37638, 50)

In [14]:
embid_dim = 50
lstm_out = 128


model = keras.Sequential()
model.add(Embedding(vocab_size, embid_dim, input_length =X.shape[1], weights = [embedding_matrix] , trainable = False))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.summary()

2023-10-08 09:29:29.222415: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-08 09:29:29.224638: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-08 09:29:29.225939: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 50)           1881900   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              183296    
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 2)                 130       
                                                        

In [15]:
batch_size = 128
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
history = model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 1, validation_data =(X_test, Y_test))

Epoch 1/5


2023-10-08 09:30:22.434108: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-08 09:30:22.436458: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-08 09:30:22.437906: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

88/88 [==============================] - ETA: 0s - loss: 0.6114 - accuracy: 0.6583

2023-10-08 09:31:29.363315: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-08 09:31:29.365142: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-08 09:31:29.366602: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

88/88 [==============================] - 78s 844ms/step - loss: 0.6114 - accuracy: 0.6583 - val_loss: 0.5208 - val_accuracy: 0.7533
Epoch 2/5
88/88 [==============================] - 74s 840ms/step - loss: 0.5298 - accuracy: 0.7499 - val_loss: 0.5595 - val_accuracy: 0.7403
Epoch 3/5
88/88 [==============================] - 74s 841ms/step - loss: 0.5007 - accuracy: 0.7715 - val_loss: 0.4593 - val_accuracy: 0.8061
Epoch 4/5
88/88 [==============================] - 73s 827ms/step - loss: 0.4820 - accuracy: 0.7752 - val_loss: 0.5083 - val_accuracy: 0.7560
Epoch 5/5
88/88 [==============================] - 72s 824ms/step - loss: 0.4914 - accuracy: 0.7737 - val_loss: 0.4690 - val_accuracy: 0.8072


In [17]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
sentences =[]
for t in  tqdm(range(len(train['text']))):
    text = nltk.word_tokenize(train['text'][t])
    sentences.append(text)

100%|██████████| 15000/15000 [00:15<00:00, 961.61it/s] 


In [21]:
!pip install gensim

  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/d3/e2/17bad124c8dd2aa0a3062e44992eb34c282379450ebbe6fdb6b96aa3c907/gensim-4.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for smart-open>=1.8.1 from https://files.pythonhosted.org/packages/fc/d9/d97f1db64b09278aba64e8c81b5d322d436132df5741c518f3823824fae0/smart_open-6.4.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 30.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 13.7 MB/s eta 0:00:00


In [22]:
import gensim

In [26]:
from gensim.models import Word2Vec

# 'sentences' should be a list of lists, where each inner list represents a sentence.
# 'size' specifies the dimensionality of the word vectors.
# 'min_count' sets the minimum count for words to be included in the vocabulary.
# 'sg' stands for Skip-gram model (sg=1) or Continuous Bag of Words (CBOW) model (sg=0).
w2v_model = Word2Vec(sentences, vector_size=300, min_count=2, sg=0)

In [27]:
words = list(w2v_model.wv.vocab)
print('Vocabulary size: %d' % len(words))

# save model 
filename = 'embedding_word2vec.txt'
w2v_model.wv.save_word2vec_format(filename, binary=False)

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [28]:
# The following line of code will obtain the vocabulary size.
vocab_size = len(w2v_model.wv)

print('Vocabulary size: %d' % vocab_size)

# Save the Word2Vec model
filename = 'embedding_word2vec.txt'
w2v_model.wv.save_word2vec_format(filename, binary=False)

Vocabulary size: 26836


In [29]:
embedding_vector = {}
f = open('./embedding_word2vec.txt')
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

26837it [00:01, 13995.26it/s]


In [31]:
embedding_matrix = np.zeros((vocab_size,300))
for word,i in tqdm(tokenizer.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None and i <vocab_size:
        embedding_matrix[i] = embedding_value    

100%|██████████| 37637/37637 [00:00<00:00, 472100.66it/s]


In [32]:
embid_dim = 300
lstm_out = 128


model = keras.Sequential()
model.add(Embedding(vocab_size, embid_dim, input_length =X.shape[1], weights = [ embedding_matrix] , trainable = False))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.summary()

2023-10-08 09:56:33.571538: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-08 09:56:33.573794: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-08 09:56:33.575404: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 300, 300)          8050800   
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              439296    
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 2)                 130       
                                                      

2023-10-08 09:56:33.773691: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_1_grad/concat/split_1/split_dim' with dtype int32
	 [[{{node gradients/split_1_grad/concat/split_1/split_dim}}]]


In [34]:
batch_size = 128
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
history = model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 1, validation_data =(X_test, Y_test))

Epoch 1/5


2023-10-08 09:58:33.601775: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-08 09:58:33.603908: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-08 09:58:33.605289: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

88/88 [==============================] - ETA: 0s - loss: 0.4597 - accuracy: 0.8011

2023-10-08 10:00:14.294935: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-08 10:00:14.296953: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-08 10:00:14.298333: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

88/88 [==============================] - 115s 1s/step - loss: 0.4597 - accuracy: 0.8011 - val_loss: 0.3972 - val_accuracy: 0.8213
Epoch 2/5
88/88 [==============================] - 111s 1s/step - loss: 0.3812 - accuracy: 0.8354 - val_loss: 0.3734 - val_accuracy: 0.8320
Epoch 3/5
88/88 [==============================] - 109s 1s/step - loss: 0.3612 - accuracy: 0.8425 - val_loss: 0.3568 - val_accuracy: 0.8533
Epoch 4/5
88/88 [==============================] - 108s 1s/step - loss: 0.3310 - accuracy: 0.8571 - val_loss: 0.3698 - val_accuracy: 0.8379
Epoch 5/5
88/88 [==============================] - 109s 1s/step - loss: 0.2858 - accuracy: 0.8789 - val_loss: 0.3156 - val_accuracy: 0.8744


In [35]:
from gensim.models import KeyedVectors
filename = 'GoogleNews-vectors-negative300.bin'
w2v_pretrained_model = KeyedVectors.load_word2vec_format(filename, binary=True) 

In [39]:
import numpy as np
from tqdm import tqdm

# Initialize embedding_matrix with zeros
embedding_matrix = np.zeros((vocab_size, 300))

# Iterate over words in tokenizer.word_index
for word, i in tqdm(tokenizer.word_index.items()):
    try:
        if i < vocab_size:  # Check if the index is within bounds
            embedding_value = w2v_pretrained_model[word]
            if embedding_value is not None:
                embedding_matrix[i] = embedding_value
    except KeyError:
        embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), 300)


100%|██████████| 37637/37637 [00:00<00:00, 170948.68it/s]


In [40]:
embid_dim = 300
lstm_out = 128


model = keras.Sequential()
model.add(Embedding(vocab_size, 300, input_length =300, weights = [embedding_matrix ] , trainable = False))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.summary()

2023-10-08 10:47:28.957735: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-08 10:47:28.959658: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-08 10:47:28.961025: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 300, 300)          8050800   
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              439296    
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dense_8 (Dense)             (None, 2)                 130       
                                                      

2023-10-08 10:47:29.159888: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_1_grad/concat/split_1/split_dim' with dtype int32
	 [[{{node gradients/split_1_grad/concat/split_1/split_dim}}]]


In [41]:
batch_size = 128
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
history = model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 1, validation_data =(X_test, Y_test))

Epoch 1/5


2023-10-08 10:47:36.849238: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-08 10:47:36.851312: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-08 10:47:36.852825: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

88/88 [==============================] - ETA: 0s - loss: 0.5454 - accuracy: 0.7164

2023-10-08 10:49:17.174808: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-08 10:49:17.176989: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-08 10:49:17.178276: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

88/88 [==============================] - 114s 1s/step - loss: 0.5454 - accuracy: 0.7164 - val_loss: 0.4256 - val_accuracy: 0.8067
Epoch 2/5
88/88 [==============================] - 108s 1s/step - loss: 0.4092 - accuracy: 0.8263 - val_loss: 0.3729 - val_accuracy: 0.8419
Epoch 3/5
88/88 [==============================] - 110s 1s/step - loss: 0.4159 - accuracy: 0.8252 - val_loss: 0.4134 - val_accuracy: 0.8213
Epoch 4/5
88/88 [==============================] - 110s 1s/step - loss: 0.3995 - accuracy: 0.8331 - val_loss: 0.3489 - val_accuracy: 0.8568
Epoch 5/5
88/88 [==============================] - 110s 1s/step - loss: 0.3615 - accuracy: 0.8493 - val_loss: 0.3719 - val_accuracy: 0.8584


In [44]:
from gensim.models import KeyedVectors
import numpy as np
from tqdm import tqdm

# Specify the path to your FastText .vec file
filename = 'wiki-news-300d-1M.vec'

# Load the FastText model
fasttext_model = KeyedVectors.load_word2vec_format(filename, binary=False)

# Initialize the embedding matrix
embedding_matrix = np.zeros((vocab_size, 300))

# Iterate over words in tokenizer.word_index
for word, i in tqdm(tokenizer.word_index.items()):
    if i < vocab_size:  # Check if the index is within bounds
        try:
            embedding_value = fasttext_model[word]
            if embedding_value is not None:
                embedding_matrix[i] = embedding_value
        except KeyError:
            embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), 300)


100%|██████████| 37637/37637 [00:00<00:00, 151073.73it/s]


In [45]:
embid_dim = 300
lstm_out = 128


model = keras.Sequential()
model.add(Embedding(vocab_size, 300, input_length =300, weights = [embedding_matrix ] , trainable = False))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.summary()

2023-10-08 11:24:14.821268: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-08 11:24:14.823081: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-08 11:24:14.825419: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 300, 300)          8050800   
                                                                 
 bidirectional_3 (Bidirectio  (None, 256)              439296    
 nal)                                                            
                                                                 
 dense_9 (Dense)             (None, 128)               32896     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dense_11 (Dense)            (None, 2)                 130       
                                                      

2023-10-08 11:24:15.036357: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-08 11:24:15.038737: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-08 11:24:15.041216: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [46]:
batch_size = 128
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
history = model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 1, validation_data =(X_test, Y_test))

Epoch 1/5


2023-10-08 11:24:24.934995: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-08 11:24:24.936848: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-08 11:24:24.938075: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

88/88 [==============================] - ETA: 0s - loss: 0.5904 - accuracy: 0.6759

2023-10-08 11:26:06.156025: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-08 11:26:06.159218: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-08 11:26:06.160612: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

88/88 [==============================] - 117s 1s/step - loss: 0.5904 - accuracy: 0.6759 - val_loss: 0.4464 - val_accuracy: 0.8248
Epoch 2/5
88/88 [==============================] - 111s 1s/step - loss: 0.5025 - accuracy: 0.7704 - val_loss: 0.4215 - val_accuracy: 0.8176
Epoch 3/5
88/88 [==============================] - 113s 1s/step - loss: 0.4306 - accuracy: 0.8188 - val_loss: 0.3798 - val_accuracy: 0.8429
Epoch 4/5
88/88 [==============================] - 111s 1s/step - loss: 0.3634 - accuracy: 0.8468 - val_loss: 0.3731 - val_accuracy: 0.8448
Epoch 5/5
88/88 [==============================] - 109s 1s/step - loss: 0.4842 - accuracy: 0.7679 - val_loss: 0.5538 - val_accuracy: 0.7032
